#**IMPORT THƯ VIỆN**

In [29]:
# Huấn luyện AI dự đoán giá nhà
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures

# **TẢI DỮ LIỆU**

In [27]:
# =======================================================
# Tải dữ liệu
# =======================================================
url = "https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv"
data = pd.read_csv(url)
print("5 DÒNG ĐẦU CỦA DỮ LIỆU:")

print("-"*100) # In ra 1 dòng trống cho dễ đọc!
print(data.head()) # In ra 5 dòng đầu tiên để kiểm tra
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

5 DÒNG ĐẦU CỦA DỮ LIỆU:
----------------------------------------------------------------------------------------------------
   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEAR BAY  
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2       496.0       177.0         7.2574            352100.0        NEAR BAY  
3       558.0       219.0         5.6431            341300.0        NEAR BAY  
4       565.0       259.0    

# **XỬ LÝ DỮ LIỆU**

##1. Xử lý Missing Value

In [28]:
# Kiểm tra Missing Value
print("KIỂM TRA MISSING VALUE:")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

print("Tổng số Missing Value theo cột: ")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!
print(data.isnull().sum())
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

print("Phần trăm Missing Value theo cột")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

print(data.isnull().mean()*100)

print("-"*100) # In ra 1 dòng trống cho dễ đọc!

# Loại bỏ Missing Value
data.dropna(inplace = True)
print("ĐÃ LOẠI BỎ MISING VALUE!")

KIỂM TRA MISSING VALUE:
----------------------------------------------------------------------------------------------------
Tổng số Missing Value theo cột: 
----------------------------------------------------------------------------------------------------
longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64
----------------------------------------------------------------------------------------------------
Phần trăm Missing Value theo cột
----------------------------------------------------------------------------------------------------
longitude             0.000000
latitude              0.000000
housing_median_age    0.000000
total_rooms           0.000000
total_bedrooms        1.002907
population            0.000000
households            0.000000
median_income   

##2. One-hot encoding cho biến phân loại

In [16]:
# One-hot encoding cho biến phân loại
data = pd.get_dummies(data, columns = ['ocean_proximity'], drop_first = True)

print('DATA SAU KHI XỬ LÝ BIẾN CATEGORICAL:')
print("-"*100) # In ra 1 dòng trống cho dễ đọc!
print(data.head())
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

DATA SAU KHI XỬ LÝ BIẾN CATEGORICAL:
----------------------------------------------------------------------------------------------------
   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value  \
0       322.0       126.0         8.3252            452600.0   
1      2401.0      1138.0         8.3014            358500.0   
2       496.0       177.0         7.2574            352100.0   
3       558.0       219.0         5.6431            341300.0   
4       565.0       259.0         3.8462            342200.0   

   ocean_proximity_INLAND 

## 3. Thêm đặc trưng dữ liệu

In [17]:
# Thêm đặc trưng dữ liệu
data['rooms_per_household'] = data['total_rooms'] /data['households']
data['bedrooms_per_room'] = data['total_bedrooms'] / data['total_rooms']
data['population_per_household'] = data['population'] / data['households']
data['rooms_per_person'] = data['total_rooms'] / data['population']
data['bedrooms_per_household'] = data['total_bedrooms'] / data['households']
data['bedroom_to_income_ratio'] = data['total_bedrooms'] / data['median_income']

print("ĐÃ THÊM ĐẶC TRƯNG DỮ LIỆU!")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

ĐÃ THÊM ĐẶC TRƯNG DỮ LIỆU!
----------------------------------------------------------------------------------------------------


## 4. Phân tích phân phối dữ liệu


In [18]:
print("PHÂN TÍCH PHÂN PHỐI DỮ LIỆU:")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

#Xây dựng hàm phân tích phân phối
def analize_distribution(df, col, plot_ = False):
    x = df[col]

    skewness = skew(x) #Độ lệch: > 0 thì lệch phải | <0 thì lệch trái
    kurt = kurtosis(x) #Độ nhọn: > 0 thì đỉnh nhọn hơn chuẩn     |      < 0 thì đỉnh bẹt hơn chuẩn

    print(f"Phân tích cột: {col}")
    print(f"Độ lệch là: {skewness}")
    print(f"Độ nhọn là: {kurt}")

    # Gợi ý xử lý độ lệch: 0

    if skewness > 3:
        sug = 'Lệch phải mạnh -> Dùng log hoặc sqrt hoặc **0.3'
        k = 1
    elif skewness > 1.5:
        sug = 'Lệch phải nhẹ -> Dùng sqrt'
        k = 2
    elif skewness <-3:
        sug = 'Lệch trái mạnh -> Dùng log hoặc **0.3 hoặc sqrt'
        k = 3
    elif skewness < -1.5:
        sug = 'Lệch trái nhẹ -> dùng sqrt'
        k = 4
    else:
        sug = ' Dữ liệu khá cân đối -> có thể giữ nguyên'
        k = 5

    print(sug)

    # Gợi ý xử lý độ nhọn:
    if kurt > 3:
        sug1 = 'Đỉnh nhọn -> có thể có outlier -> nên kiểm tra ngoại lệ!'
        h = 1
    elif kurt < -3:
        sug1 = 'Đỉnh bẹt -> Dữ liệu phân tán, có thể chuẩn hóa'
        h = 2
    else:
        sug1 = 'Dữ liệu Phân phối chuẩn'
        h = 3

    print(sug1)
    print("-"*100) # In ra 1 dòng trống cho dễ đọc!

    # Nếu muốn in ra biểu đồ để quan sát độ lệch và độ nhọn dữ liệu => Mở phong ấn đoạn này :)))

    # Vẽ biểu đồ:
    if plot_:
        plt.figure(figsize = (10, 4)) # Tạo biểu đồ mới: rộng 10, cao 4
        sns.histplot(x, kde=True, bins = 30)
        plt.title(f"Độ lệch của {col} là {skewness:.3f}")
        plt.xlabel(col)
        plt.show()
    return skewness, kurt

skews = []
kurtosiss = []
dict_pp = {}
for col in data.columns:
    if 'ocean_proximity' in col:
        continue
    distr = analize_distribution(data, col)
    dict_pp[col] = distr
    #key: col | value : (skew, kurt)

# Lập danh sách các cột cần xử lý
col_ana = []

for key, value in dict_pp.items():
    sk, kt = value
    if sk > 0.5 or sk < -0.5:
        col_ana.append(key)
        continue

    if kt > 2 or kt < -2:
        col_ana.append(key)
        continue

# Xây dựng hàm xử lý outlier: IQR
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25) # tìm giá trị Q1
    Q3 = df[column].quantile(0.75) # tìm giá trị Q3
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR  # Xác định ngưỡng dưới
    upper_bound = Q3 + 1.5 * IQR  # Xác định ngưỡng trên

    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

    print(f"📌 Đã loại bỏ {len(df) - len(filtered_df)} outliers ở cột '{column}'")
    return filtered_df

print(f"Kích thước bộ dữ liệu trước xử lý: {data.shape}")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

# Xử lý phân phối dữ liệu
for col in dict_pp.keys():

    # xử lý độ lệch
    if abs(dict_pp[col][0])  > 5:
        data[col] = np.log1p(data[col])

    elif abs(dict_pp[col][0]) > 3:
        data[col] = data[col]**0.3

    elif abs(dict_pp[col][0]) > 1:
        data[col] = data[col]**0.5

    # Xử lý độ nhọn
    if dict_pp[col][1] > 5:
        data = remove_outliers_iqr(data, col)
        continue
    if dict_pp[col][1] < -5:
        scaler = StandardScaler()
        data[col] = scaler.fit_transform(data[col])

print("-"*100) # In ra 1 dòng trống cho dễ đọc!

PHÂN TÍCH PHÂN PHỐI DỮ LIỆU:
----------------------------------------------------------------------------------------------------
Phân tích cột: longitude
Độ lệch là: -0.29611916023301665
Độ nhọn là: -1.33251572728317
 Dữ liệu khá cân đối -> có thể giữ nguyên
Đỉnh bẹt -> Dữ liệu phân tán, có thể chuẩn hóa
----------------------------------------------------------------------------------------------------
Phân tích cột: latitude
Độ lệch là: 0.4649001451840516
Độ nhọn là: -1.119542249537143
 Dữ liệu khá cân đối -> có thể giữ nguyên
Đỉnh bẹt -> Dữ liệu phân tán, có thể chuẩn hóa
----------------------------------------------------------------------------------------------------
Phân tích cột: housing_median_age
Độ lệch là: 0.061600903242716894
Độ nhọn là: -0.8011109750230592
 Dữ liệu khá cân đối -> có thể giữ nguyên
Đỉnh bẹt -> Dữ liệu phân tán, có thể chuẩn hóa
----------------------------------------------------------------------------------------------------
Phân tích cột: total_rooms


<ipython-input-18-5b069bb3e85c>:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col]**0.3


##5. Sinh đặc trưng phi tuyến bậc 3

In [19]:
# Lấy nhãn y
y = data['median_house_value']

# Xóa nhãn khỏi data để tránh lộ dữ liệu
data = data.drop(columns=['median_house_value'])

# Sinh đặc trưng phi tuyến bậc 3 và không thêm bias do Linear Regression mạc định có rồi!
poly = PolynomialFeatures(degree=3, include_bias=False)
data = poly.fit_transform(data)
print('ĐÃ SINH ĐẶC TRƯNG PHI TUYẾN BẬC 3!')
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

# Ép lại Data thành Dataframe để dễ thao tác!
data = pd.DataFrame(data, columns=poly.get_feature_names_out())

# Lấy dữ liệu huấn luyện X
X = data

# Check độ lệch và độ nhọn của nhãn y xem cần xử lý không!
sk = skew(y)
kt = kurtosis(y)

# In ra độ lệch và độ nhọn để quan sát!
print(f"Độ lệch của nhãn là: {sk}")
print(f"Độ nhọn của nhãn là: {kt}")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

ĐÃ SINH ĐẶC TRƯNG PHI TUYẾN BẬC 3
----------------------------------------------------------------------------------------------------
Độ lệch của nhãn là: 0.9309261297277046
Độ nhọn của nhãn là: 0.24543950520048075
----------------------------------------------------------------------------------------------------


## 6. Xử lý NaN sau khi sinh đặc trưng phi tuyến

In [20]:
# Xóa các cột có toàn bộ là NaN
X = X.dropna(axis = 1, how = 'all')
print("-"*100) # In ra 1 dòng trống cho dễ đọc!
print("Kích thước bộ dữ liệu sau khi xóa các cột all NaN")
print(X.shape) # In ra kích thước bộ dữ liệu xem có xóa quá nhiều không!

# In ra các cột vẫn còn NaN (để xử lý tiếp)
cols_with_nan = X.columns[X.isna().any()]

print("-"*100) # In ra 1 dòng trống cho dễ đọc!
print("Cột còn thiếu giá trị:", cols_with_nan)

# Ví dụ: điền NaN bằng median cho tất cả các cột còn thiếu (cách nhanh gọn)
for col in cols_with_nan:
    if X[col].dtype in ['float64', 'int64']:
        X[col] = X[col].fillna(X[col].median())
    else:
        X[col] = X[col].fillna(X[col].mode()[0])

# In ra kích thước sau khi xử lý NaN
print("-"*100) # In ra 1 dòng trống cho dễ đọc!
print("Kích thước sau khi xử lý NaN lần cuối: ")
print(X.shape)
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

----------------------------------------------------------------------------------------------------
Kích thước bộ dữ liệu sau khi xóa các cột all NaN
(16893, 1329)
----------------------------------------------------------------------------------------------------
Cột còn thiếu giá trị: Index([], dtype='object')
----------------------------------------------------------------------------------------------------
Kích thước sau khi xử lý NaN lần cuối: 
(16893, 1329)
----------------------------------------------------------------------------------------------------


## 7. Chia tập train/test

In [22]:
# Chia thành tập train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()

# Đoạn này tôi thấy tắt chuẩn hóa đi cho R2 Score cao hơn nên đã phong ấn nó!

# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)


# In ra kích thước của bộ dữ liệu trước khi train: Check lần cuối
print("KIỂM TRA SƠ BỘ DỮ LIỆU LẦN CUỐI: ")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

print("Số dòng dữ liệu train:", X_train.shape[0])
print("Số dòng dữ liệu test:", X_test.shape[0])
print("Số dòng tổng cộng sau xử lý:", data.shape[0])
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

KIỂM TRA SƠ BỘ DỮ LIỆU LẦN CUỐI: 
----------------------------------------------------------------------------------------------------
Số dòng dữ liệu train: 13514
Số dòng dữ liệu test: 3379
Số dòng tổng cộng sau xử lý: 16893
----------------------------------------------------------------------------------------------------


# **HUẤN LUYỆN MÔ HÌNH**

In [30]:
# ================================================================
# Huấn luyện mô hình
# ================================================================
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

# **ĐÁNH GIÁ MÔ HÌNH**

In [24]:
# ================================================================
# Đánh giá mô hình
# ================================================================

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    #Sai số trung bình tuyệt đối
    mae = mean_absolute_error(y_test, y_pred)

    # Sai số bình phương trung bình
    mse = mean_squared_error(y_test, y_pred)

    # Sai số phần trăm trung bình tuyệt đối
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

   # Độ chính xác theo R2
    r2 = r2_score(y_test, y_pred)

    print(f"{model.__class__.__name__}")
    print(f"  - MAE: {mae:.4f}")
    print(f"  - MSE: {mse:.4f}")
    print(f"  - R² Score: {r2:.4f}")
    print(f"  - MAPE: {mape:.2f}%")  # Tính sai số phần trăm
    print("-"*100) # In ra 1 dòng trống cho dễ đọc!

print("ĐÁNH GIÁ MÔ HÌNH:")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

evaluate_model(lr_model, X_test, y_test)

ĐÁNH GIÁ MÔ HÌNH:
----------------------------------------------------------------------------------------------------
LinearRegression
  - MAE: 35523.6148
  - MSE: 2629727026.5873
  - R² Score: 0.8021
  - MAPE: 19.98%
----------------------------------------------------------------------------------------------------


# **ĐOẠN CODE TỔNG THỂ**
###(ĐÃ LÀM SẠCH SƠ BỘ - CÓ THỂ UPDATE)

In [12]:
# Huấn luyện AI dự đoán giá nhà
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures

# =======================================================
# Tải dữ liệu
# =======================================================
url = "https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv"
data = pd.read_csv(url)
print("5 DÒNG ĐẦU CỦA DỮ LIỆU:")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!
print(data.head()) # In ra 5 dòng đầu tiên để kiểm tra
print("-"*100) # In ra 1 dòng trống cho dễ đọc!
# ================================================================
# Xử lý dữ liệu
# ================================================================

# ================================================================
# Level 1:
# ================================================================

# Kiểm tra Missing Value
print("KIỂM TRA MISSING VALUE:")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

print("Tổng số Missing Value theo cột: ")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!
print(data.isnull().sum())

print("-"*100) # In ra 1 dòng trống cho dễ đọc!
print("Phần trăm Missing Value theo cột")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!
print(data.isnull().mean()*100)

print("-"*100) # In ra 1 dòng trống cho dễ đọc!

# Loại bỏ Missing Value
data.dropna(inplace = True)

# One-hot encoding cho biến phân loại
data = pd.get_dummies(data, columns = ['ocean_proximity'], drop_first = True)
print('DATA SAU KHI XỬ LÝ BIẾN CATEGORICAL:')
print("-"*100) # In ra 1 dòng trống cho dễ đọc!
print(data.head())
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

# Thêm đặc trưng dữ liệu
data['rooms_per_household'] = data['total_rooms'] /data['households']
data['bedrooms_per_room'] = data['total_bedrooms'] / data['total_rooms']
data['population_per_household'] = data['population'] / data['households']
data['rooms_per_person'] = data['total_rooms'] / data['population']
data['bedrooms_per_household'] = data['total_bedrooms'] / data['households']
data['bedroom_to_income_ratio'] = data['total_bedrooms'] / data['median_income']

print("ĐÃ THÊM ĐẶC TRƯNG DỮ LIỆU!")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

print("PHÂN TÍCH PHÂN PHỐI DỮ LIỆU:")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

#Xây dựng hàm phân tích phân phối
def analize_distribution(df, col, plot_ = False):
    x = df[col]

    skewness = skew(x) #Độ lệch: > 0 thì lệch phải | <0 thì lệch trái
    kurt = kurtosis(x) #Độ nhọn: > 0 thì đỉnh nhọn hơn chuẩn     |      < 0 thì đỉnh bẹt hơn chuẩn

    print(f"Phân tích cột: {col}")
    print(f"Độ lệch là: {skewness}")
    print(f"Độ nhọn là: {kurt}")

    # Gợi ý xử lý độ lệch:

    if skewness > 3:
        sug = 'Lệch phải mạnh -> Dùng log hoặc sqrt hoặc **0.3'
        k = 1
    elif skewness > 1.5:
        sug = 'Lệch phải nhẹ -> Dùng sqrt'
        k = 2
    elif skewness <-3:
        sug = 'Lệch trái mạnh -> Dùng log hoặc **0.3 hoặc sqrt'
        k = 3
    elif skewness < -1.5:
        sug = 'Lệch trái nhẹ -> dùng sqrt'
        k = 4
    else:
        sug = ' Dữ liệu khá cân đối -> có thể giữ nguyên'
        k = 5

    print(sug)

    # Gợi ý xử lý độ nhọn:
    if kurt > 4:
        sug1 = 'Đỉnh nhọn -> có thể có outlier -> nên kiểm tra ngoại lệ!'
        h = 1
    elif kurt < 2:
        sug1 = 'Đỉnh bẹt -> Dữ liệu phân tán, có thể chuẩn hóa'
        h = 2
    else:
        sug1 = 'Dữ liệu Phân phối chuẩn'
        h = 3

    print(sug1)
    print("-"*100) # In ra 1 dòng trống cho dễ đọc!

    # Nếu muốn in ra biểu đồ để quan sát độ lệch và độ nhọn dữ liệu => Mở phong ấn đoạn này :)))

    # Vẽ biểu đồ:
    if plot_:
        plt.figure(figsize = (10, 4)) # Tạo biểu đồ mới: rộng 10, cao 4
        sns.histplot(x, kde=True, bins = 30)
        plt.title(f"Độ lệch của {col} là {skewness:.3f}")
        plt.xlabel(col)
        plt.show()
    return skewness, kurt

skews = []
kurtosiss = []
dict_pp = {}
for col in data.columns:
    if 'ocean_proximity' in col:
        continue
    distr = analize_distribution(data, col)
    dict_pp[col] = distr

# Lập danh sách các cột cần xử lý
col_ana = []

for key, value in dict_pp.items():
    sk, kt = value
    if sk > 0.5 or sk < -0.5:
        col_ana.append(key)
        continue

    if kt > 2 or kt < -2:
        col_ana.append(key)
        continue

# Xây dựng hàm xử lý outlier:
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25) # tìm giá trị Q1
    Q3 = df[column].quantile(0.75) # tìm giá trị Q3
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR  # Xác định ngưỡng dưới
    upper_bound = Q3 + 1.5 * IQR  # Xác định ngưỡng trên

    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

    print(f"📌 Đã loại bỏ {len(df) - len(filtered_df)} outliers ở cột '{column}'")
    return filtered_df

print(f"Kích thước bộ dữ liệu trước xử lý: {data.shape}")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

# Xử lý phân phối dữ liệu
for col in dict_pp.keys():

    # xử lý độ lệch
    if abs(dict_pp[col][0])  > 5:
        data[col] = np.log1p(data[col])

    elif abs(dict_pp[col][0]) > 3:
        data[col] = data[col]**0.3

    elif abs(dict_pp[col][0]) > 1:
        data[col] = data[col]**0.5

    # Xử lý độ nhọn
    if dict_pp[col][1] > 5:
        data = remove_outliers_iqr(data, col)
        continue
    if dict_pp[col][1] < -5:
        scaler = StandardScaler()
        data[col] = scaler.fit_transform(data[col])

print("-"*100) # In ra 1 dòng trống cho dễ đọc!
# ================================================================
# Level 2:
# ================================================================

# Lấy nhãn y
y = data['median_house_value']

# Xóa nhãn khỏi data để tránh lộ dữ liệu
data = data.drop(columns=['median_house_value'])

# Sinh đặc trưng phi tuyến bậc 3 và không thêm bias do Linear Regression mạc định có rồi!
poly = PolynomialFeatures(degree=3, include_bias=False)
data = poly.fit_transform(data)

# Ép lại Data thành Dataframe để dễ thao tác!
data = pd.DataFrame(data, columns=poly.get_feature_names_out())

# Lấy dữ liệu huấn luyện X
X = data

# Check độ lệch và độ nhọn của nhãn y xem cần xử lý không!
sk = skew(y)
kt = kurtosis(y)

# In ra độ lệch và độ nhọn để quan sát!
print(f"Độ lệch của nhãn là: {sk}")
print(f"Độ nhọn của nhãn là: {kt}")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

# ================================================================
# Level 2.5: Xử lý NaN sau khi sinh đặc trưng phi tuyến
# ================================================================

# Xóa các cột có toàn bộ là NaN
X = X.dropna(axis = 1, how = 'all')
print("-"*100) # In ra 1 dòng trống cho dễ đọc!
print("Kích thước bộ dữ liệu sau khi xóa các cột all NaN")
print(X.shape) # In ra kích thước bộ dữ liệu xem có xóa quá nhiều không!

# In ra các cột vẫn còn NaN (để xử lý tiếp)
cols_with_nan = X.columns[X.isna().any()]

print("-"*100) # In ra 1 dòng trống cho dễ đọc!
print("Cột còn thiếu giá trị:", cols_with_nan)

# Ví dụ: điền NaN bằng median cho tất cả các cột còn thiếu (cách nhanh gọn)
for col in cols_with_nan:
    if X[col].dtype in ['float64', 'int64']:
        X[col] = X[col].fillna(X[col].median())
    else:
        X[col] = X[col].fillna(X[col].mode()[0])

# In ra kích thước sau khi xử lý NaN
print("-"*100) # In ra 1 dòng trống cho dễ đọc!
print("Kích thước sau khi xử lý NaN lần cuối: ")
print(X.shape)
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

# Chia thành tập train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()

# Đoạn này tôi thấy tắt chuẩn hóa đi cho R2 Score cao hơn nên đã phong ấn nó!

# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)


# In ra kích thước của bộ dữ liệu trước khi train: Check lần cuối
print("Kiểm tra bộ dữ liệu lần cuối: ")
print("Số dòng dữ liệu train:", X_train.shape[0])
print("Số dòng dữ liệu test:", X_test.shape[0])
print("Số dòng tổng cộng sau xử lý:", data.shape[0])
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

# ================================================================
# Huấn luyện mô hình
# ================================================================
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# ================================================================
# Đánh giá mô hình
# ================================================================

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    #Sai số trung bình tuyệt đối
    mae = mean_absolute_error(y_test, y_pred)

    # Sai số bình phương trung bình
    mse = mean_squared_error(y_test, y_pred)

    # Sai số phần trăm trung bình tuyệt đối
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

   # Độ chính xác theo R2
    r2 = r2_score(y_test, y_pred)

    print(f"{model.__class__.__name__}")
    print(f"  - MAE: {mae:.4f}")
    print(f"  - MSE: {mse:.4f}")
    print(f"  - R² Score: {r2:.4f}")
    print(f"  - MAPE: {mape:.2f}%")  # Tính sai số phần trăm
    print("-"*100) # In ra 1 dòng trống cho dễ đọc!

print("ĐÁNH GIÁ MÔ HÌNH:")
print("-"*100) # In ra 1 dòng trống cho dễ đọc!

evaluate_model(lr_model, X_test, y_test)


5 DÒNG ĐẦU CỦA DỮ LIỆU:
----------------------------------------------------------------------------------------------------
   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEAR BAY  
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2       496.0       177.0         7.2574            352100.0        NEAR BAY  
3       558.0       219.0         5.6431            341300.0        NEAR BAY  
4       565.0       259.0    